In [3]:
pip install keras tensorflow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.4/217.4 MB 643.3 kB/s eta 0:00:0000:0100:10
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.3/4.3 MB 498.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.7/126.7 KB 551.9 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.5/462.5 KB 268.2 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 422.3 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.0/13.0 MB 942.6 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.3/962.3 KB 376.1 kB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 KB 148.0 kB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 807.7 kB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 KB 1.1 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.5/57.5 

  Running setup.py install for termcolor ... done
Note: you may need to restart the kernel to use updated packages.


In [1]:
import keras

In [2]:
from src.core import DataLoader

In [3]:
dl = DataLoader('data/jokes.csv', 'data/rating.csv')

In [4]:
dl.train_data

,id,user_id,joke_id,Rating
0,31030_110,31030,110,2.750
2,23098_6,23098,6,-6.438
3,14273_86,14273,86,4.406
4,18419_134,18419,134,9.375
5,5782_14,5782,14,-1.781
...,...,...,...,...
1092053,4860_62,4860,62,9.000
1092054,9517_132,9517,132,3.156
1092055,27767_118,27767,118,-1.594
1092056,10580_81,10580,81,2.000


In [5]:
import numpy as np

In [6]:
users_train = {}
users_train_y = {}

for record in dl.train_data.iterrows():
    r = record[1]
    
    if r['user_id'] not in users_train:
        users_train[r['user_id']] = np.ones(139) * 0.5
        users_train_y[r['user_id']] = np.ones(139) * -1
    
    users_train[r['user_id']][r['joke_id'] - 1] = (r['Rating'] + 10) / 20
    users_train_y[r['user_id']][r['joke_id'] - 1] = (r['Rating'] + 10) / 20


In [7]:
users_test = {}
users_test_y = {}

for record in dl.test_data.iterrows():
    r = record[1]
    
    if r['user_id'] not in users_test:
        users_test[r['user_id']] = np.ones(139) * 0.5
        users_test_y[r['user_id']] = np.ones(139) * -1
    
    users_test[r['user_id']][r['joke_id'] - 1] = (r['Rating'] + 10) / 20
    users_test_y[r['user_id']][r['joke_id'] - 1] = (r['Rating'] + 10) / 20


In [8]:
x_train = np.array(list(users_train.values()))
y_train = np.array(list(users_train_y.values()))
x_test = np.array(list(users_test.values()))
y_test = np.array(list(users_test_y.values()))

In [9]:
ind = np.where(y_train[0] > -1)[0]
x_train[y_train > -1]

array([0.1297 , 0.14375, 0.03125, ..., 0.99845, 0.7047 , 0.1375 ])

In [10]:
import tensorflow as K

def custom_loss(y_true, y_pred):          
    ind = K.where(y_true + 1)
    sliced_p=K.gather_nd(y_pred, indices=ind)
    sliced_t = K.gather_nd(y_true, indices=ind)
    
    sqr = K.square(sliced_p - sliced_t)
    
    return K.sqrt(K.reduce_mean(sqr))

In [11]:
from keras import layers

In [69]:
inp = keras.Input(shape=(139,))
encoded = layers.Dense(64, activation='relu')(inp)
encoded = layers.Dense(32, activation='relu')(encoded)
encoded = layers.Dense(16, activation='relu')(encoded)

decoded = layers.Dense(32, activation='relu')(encoded)
decoded = layers.Dense(64, activation='relu')(decoded)
decoded = layers.Dense(139, activation='sigmoid')(decoded)

In [70]:
autoencoder = keras.Model(inp, decoded)
autoencoder.compile(optimizer='adam', loss=custom_loss)

autoencoder.fit(x_train, y_train,
                epochs=50,
                batch_size=32,
                shuffle=True,
                validation_data=(x_test, y_test))

Epoch 1/50
1277/1277 [==============================] - 3s 2ms/step - loss: 0.2090 - val_loss: 0.2132
Epoch 2/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1865 - val_loss: 0.1979
Epoch 3/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1810 - val_loss: 0.1883
Epoch 4/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1781 - val_loss: 0.1809
Epoch 5/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1757 - val_loss: 0.1819
Epoch 6/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1745 - val_loss: 0.1819
Epoch 7/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1735 - val_loss: 0.1818
Epoch 8/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1724 - val_loss: 0.1875
Epoch 9/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.1714 - val_loss: 0.1753
Epoch 10/50
1277/1277 [==============================] - 2s 2ms/step - loss: 0.169

In [151]:
users_train[31030][users_train_y[31030] != -1]

array([0.1297 , 0.14375, 0.7547 , 0.9281 , 0.01405, 0.1547 , 0.7219 ,
       0.20155, 0.6297 , 0.7344 , 0.5344 , 0.01095, 0.575  , 0.0422 ,
       0.725  , 0.0578 , 0.39845, 0.55155, 0.95625, 0.1344 , 0.7297 ,
       0.0594 , 0.93125, 0.29845, 0.7953 , 0.03125, 0.26095, 0.0125 ,
       0.025  , 0.9578 , 0.02345, 0.57345, 0.3781 , 0.06405, 0.1172 ,
       0.34375, 0.0172 , 0.2203 , 0.7219 , 0.6    , 0.7453 , 0.1031 ,
       0.23125, 0.8844 , 0.075  , 0.1297 , 0.60625, 0.0172 , 0.21405,
       0.225  , 0.2531 , 0.6594 , 0.6375 , 0.18125, 0.1844 , 0.7094 ,
       0.875  , 0.34375, 0.6328 , 0.9047 , 0.3172 , 0.3094 , 0.0656 ,
       0.02345, 0.0422 , 0.21095, 0.01405, 0.69375])

In [38]:
autoencoder.predict(users_train[31030].reshape((1, 139)))[0][users_train_y[31030] != -1]

array([0.07454482, 0.18807223, 0.36238152, 0.31375545, 0.24415049,
       0.37008524, 0.3264488 , 0.4211457 , 0.3112281 , 0.35130882,
       0.31420618, 0.28400612, 0.4894547 , 0.24763054, 0.28317586,
       0.34223348, 0.43269318, 0.28422183, 0.36677742, 0.52247685,
       0.44576147, 0.4305954 , 0.3670886 , 0.4268012 , 0.31815875,
       0.5545485 , 0.35171258, 0.3654453 , 0.49993196, 0.3835779 ,
       0.28393063, 0.28888196, 0.24497369, 0.37615705, 0.28186947,
       0.37613627, 0.3012725 , 0.5939765 , 0.34360278, 0.41193992,
       0.36981803, 0.47237232, 0.4311819 , 0.2804407 , 0.39539552,
       0.50345   , 0.3618981 , 0.3572039 , 0.40525812, 0.22167814,
       0.14630115, 0.32040215, 0.39908135, 0.24789602, 0.32041878,
       0.34500068, 0.32850912, 0.54408854, 0.21772864, 0.30286634,
       0.21447319, 0.41089106, 0.34410185, 0.316423  ], dtype=float32)

In [71]:
encd = keras.Model(inp, encoded)

In [72]:
encd.predict(users_train[31030].reshape((1, 139)))

array([[1.3216784 , 0.758338  , 2.174471  , 0.        , 0.        ,
        0.25169867, 0.3181636 , 1.3698056 , 1.0705769 , 1.3650913 ,
        0.91003203, 1.5636183 , 0.8822842 , 0.        , 0.        ,
        0.        ]], dtype=float32)

In [73]:
pop = dl.train_data.groupby('joke_id').mean()

In [74]:
pdd = encd.predict(x_train)

In [75]:
test = dl.test_data

In [76]:
user_profiles = {}

for user_id in users_train.keys():
    user_profiles[user_id] = pdd[user_id - 1, :]

In [77]:
from scipy.sparse import csr_matrix

df = dl.train_data

N = len(df['user_id'].unique())
M = len(df['joke_id'].unique())

user_mapper = dict(zip(np.unique(df["user_id"]), list(range(N))))
movie_mapper = dict(zip(np.unique(df["joke_id"]), list(range(M))))

user_inv_mapper = dict(zip(list(range(N)), np.unique(df["user_id"])))
movie_inv_mapper = dict(zip(list(range(M)), np.unique(df["joke_id"])))

user_index = [user_mapper[i] for i in df['user_id']]
movie_index = [movie_mapper[i] for i in df['joke_id']]

X = csr_matrix((df["Rating"], (movie_index, user_index)), shape=(M, N))

In [78]:
x_train[0:1,:].shape

(1, 139)

In [84]:
import math
from scipy.spatial import distance

err = 0
err_dl = 0
err_pop = 0

for r in test[:100].iterrows():
    joke_id = r[1]['joke_id']
    user_id = r[1]['user_id']

    sm = 0
    wg = 0

    C = X[joke_id - 1, :].toarray()[0, :]
    D = C.nonzero()[0]
    
    for ind in D.tolist():
        sim = abs(1 / distance.cosine(user_profiles[ind + 1], user_profiles[user_id]))
        sm += sim * X[joke_id - 1, ind]
        wg += sim
    
    pred_dl = (autoencoder.predict(x_train[user_id - 1:user_id,:])[0, joke_id - 1] * 20) - 10
    pred = sm / wg
    err += abs(pred - r[1]['Rating'])
    err_pop += abs(pop.loc[joke_id]['Rating'] - r[1]['Rating'])
    err_dl += abs(pred_dl - r[1]['Rating'])

In [88]:
err / 100

3.919513851623659

In [89]:
err_pop / 100

3.9231943658384334

In [90]:
err_dl / 100

4.457352735786437